In [2]:
import pandas as pd
import numpy as np

In [3]:
from collections import Counter

In [4]:
import pandas as pd
import json
import keras
import numpy as np
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout,\
                         TimeDistributed, Concatenate, Dense, GRU, Conv1D,\
                         LeakyReLU,concatenate,GlobalMaxPool1D
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
# from CrfLayer import CRF
from keras_contrib.layers import CRF


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True
session = tf.Session(config=tfconfig)




    

def word2id(texts,MAXLEN):


  tokenizer = Tokenizer(num_words=None,) # 分词MAX_NB_WORDS
  tokenizer.fit_on_texts(texts)
  X_train_word_ids = tokenizer.texts_to_sequences(texts) #受num_words影响
  x_data=pad_sequences(X_train_word_ids,maxlen=MAXLEN,truncating='post',padding='post',value=0)

  n_vocab_char = len(tokenizer.index_word)+1
 
  return x_data,n_vocab_char


def text2id(x,col='content'):
    content =x[col]
    content_label =len(content) *[1]
    content_label = np.array(content_label)
    try:
       
        pattern =x['entity'].replace(' ','|')
        it = re.finditer(pattern,content) 
        for match in it: 
          start,end = match.span()
          if(start==end):
              continue
          content_label[start]=2
          content_label[(start+1):end]=3
    except:
        pass
    return list(content_label)


def decoder(text_ids,texts):
  final_res =[]
  for i in range(len(text_ids)):
      text = texts[i]
      text_id =text_ids[i]
      res = ''
      for j in range(len(text_id)):
          if(text_id[j]==1 or text_id[j]==0):
              pass
          else:
              if(text_id[j])==2:
                  res =res+' '+text[j]
              else:
                  res=res+text[j]
      final_res.append(res)
  return final_res


from collections import Counter
def post(x):
    #格式处理：c
    x = np.array(x).flatten().tolist()
    x=' '.join(x)
    x = x.split() 
    #过滤

    return[i[0] for i in  Counter(x).most_common(3)]

def postposs(df):
  '''

  聚合结果
  '''

  df.fillna(' ',inplace=False)
  print(list(df))
  feat_agg = ['entity_pred','entity',]
  gg = df.groupby(['newsId'])
  df_agg = gg[feat_agg[0]].apply(lambda x:' '.join(x)).reset_index()
  for f in feat_agg:
      df_agg[f] = gg[f].apply(list).reset_index()[f]
  df_agg['entity']= df_agg['entity'].map(lambda x:x[0])

  print(list(df_agg))

  return df_agg 





Using TensorFlow backend.


In [5]:
out_path='../data/coreEntityEmotion_train.txt.agg.pick'
df_new=pd.read_pickle(out_path)
#agg新闻太长把句子分开。

In [6]:
df_new.head()

,newsId,texts,entity,emotion
0,4e36d02a,sia智慧工厂展，誉洋以“智”取胜。第十七届上海国际工业自动化及机器人展与上海智能工厂展览会...,3d 工业 机器视觉,POS POS POS
1,4e36d02a,业智能设备的研发、生产制造和服务，创新理念伴随企业不断成长。今天的誉洋已与中国多家企业达成合...,3d 工业 机器视觉,POS POS POS
2,cb8e8b79,大数据可视化应用领域探析。大数据之热度，已无需多言。业内众多关于大数据可视化应用领域的声音与...,数据 卫星 可视化,POS POS POS
3,cb8e8b79,分布与运行态势情况。案例2：卫星分布运行可视化通过将宇宙空间内所有卫星的运行数据进行可视化展...,数据 卫星 可视化,POS POS POS
4,cb8e8b79,和监测。应用领域三、数据统计分析可视化此领域是目前媒体大众提及最多的应用，可用于商业智能、政...,数据 卫星 可视化,POS POS POS


In [7]:
df_new['labels']=df_new.apply(lambda x:text2id(x,col='texts'),axis=1)

In [9]:
df_new.head()

,newsId,texts,entity,emotion,labels
0,4e36d02a,sia智慧工厂展，誉洋以“智”取胜。第十七届上海国际工业自动化及机器人展与上海智能工厂展览会...,3d 工业 机器视觉,POS POS POS,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,4e36d02a,业智能设备的研发、生产制造和服务，创新理念伴随企业不断成长。今天的誉洋已与中国多家企业达成合...,3d 工业 机器视觉,POS POS POS,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,cb8e8b79,大数据可视化应用领域探析。大数据之热度，已无需多言。业内众多关于大数据可视化应用领域的声音与...,数据 卫星 可视化,POS POS POS,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,cb8e8b79,分布与运行态势情况。案例2：卫星分布运行可视化通过将宇宙空间内所有卫星的运行数据进行可视化展...,数据 卫星 可视化,POS POS POS,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,cb8e8b79,和监测。应用领域三、数据统计分析可视化此领域是目前媒体大众提及最多的应用，可用于商业智能、政...,数据 卫星 可视化,POS POS POS,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [10]:
texts=df_new['texts'].map(lambda x:list(x)).values

In [18]:
parms={} 
parms['MAXLEN']=40 

In [19]:
texts_id ,parms['n_vocab_char']= word2id(texts,parms['MAXLEN'])

In [20]:
parms['n_vocab_char']

8273

In [24]:
texts_id.shape
#多少行多少个样本

(45081, 40)

In [39]:
text_id

NameError: name 'text_id' is not defined

In [26]:
labels = df_new['labels'].values

In [29]:
labels_id=pad_sequences(labels,maxlen=parms['MAXLEN'],truncating='post',padding='post',value=0)

In [30]:
labels_id

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)

In [31]:

df_new['labels']=df_new.apply(lambda x:text2id(x,col='texts'),axis=1)

texts=df_new['texts'].map(lambda x:list(x)).values
texts_id ,parms['n_vocab_char']= word2id(texts,parms['MAXLEN'])

labels = df_new['labels'].values
labels_id=pad_sequences(labels,maxlen=parms['MAXLEN'],truncating='post',padding='post',value=0)

print('labels_id shape',labels_id.shape)
print('texts_id',texts_id.shape)

labels_id shape (45081, 40)
texts_id (45081, 40)


In [22]:
df_new.iloc[14].entity                                             

'数据 卫星 可视化'

In [18]:
s =df_new.iloc[14].texts      

In [ ]:
数据 卫星 可视化

In [23]:
e = '数据 卫星 可视化'

In [24]:
s

'数据传输等等。除此之外，对卫星回传的数据，卫星自身的状态，也有针对性的可视化分析和监测。应用领域三、数据统计分析可视化此领域是目前媒体大众提及最多的应用，可用于商业智能、政府决策、公众服务、市场营销等领域。1.商业智能可视化通过采集相关数据'

In [33]:
import re

In [34]:
content =s
content_label =len(content) *[-1]
content_label = np.array(content_label)


pattern =e.replace(' ','|')
it = re.finditer(pattern,content) 
for match in it: 
    start,end = match.span()
    if(start==end):
      continue
    content_label[start]=2
    content_label[(start+1):end]=3



In [36]:
#list(content_label)

In [37]:
#df_new

In [33]:
feat_agg = ['emotion','entity',]
gg = df_new.groupby(['newsId'])
df_agg = gg[feat_agg[0]].apply(lambda x:' '.join(x)).reset_index()
for f in feat_agg:
    df_agg[f] = gg[f].apply(list).reset_index()[f]
df_agg['entity']= df_agg['entity'].map(lambda x:x[0])


In [ ]:
def postposs(df):
  '''

  聚合结果
  '''

  df.fillna(' ',inplace=False)
  print(list(df))
  feat_agg = ['entity_pred','entity']
  gg = df.groupby(['newsId'])
  df_agg = gg[feat_agg[0]].apply(lambda x:' '.join(x)).reset_index()
  for f in feat_agg:
      df_agg[f] = gg[f].apply(list).reset_index()[f]
  df_agg['entity']= df_agg['entity'].map(lambda x:x[0])

  print(list(df_agg))

  return df_agg 

In [25]:
def post(x):
    #格式处理：
    x = np.array(x).flatten().tolist()
    x=' '.join(x)
    x = x.split() 
    #过滤

    return[i[0] for i in  Counter(x).most_common(3)]
#取出现次数最长的当预测结果

In [26]:
df_agg['predict'] = df_agg['predict'].map(lambda x:post(x))
  

In [27]:
df_agg

,newsId,predict,entity
0,613e366f,"[[], ['黄伟栋']]",cier 春招 薪酬
1,6407142f,"[[], ['增肥'], ['两块代']]",马思纯 新戏 减肥
2,940b4eb1,"[[], ['都挺好'], ['郭京飞']]",重男轻女 都挺好 原生家庭
3,cb26f088,"[[], ['鹿晗'], ['民国']]",鹿晗 民国 表情包


In [ ]:
def ner_score(real,predict):
    for i in range(len(real)):
        tp+=len(set())